In [1]:
import triton 
import torch
import triton.language as tl

In [2]:
@triton.jit 
def matrixaddition(inptr1,inptr2,outptr,m,n,block_size1:tl.constexpr,block_size2:tl.constexpr):
    id_m=tl.program_id(0)
    id_n=tl.program_id(1)

    mstart=block_size1*id_m
    nstart=block_size2*id_n

    m_range = mstart + tl.arange(0, block_size1)
    n_range = nstart + tl.arange(0, block_size2)

   ## this is new thing which wasnt there in my earlier cod e since all old were 1d arrays
    m_offsets=m_range[:,None]
    n_offsets=n_range[None,:]

    m_mask=m_offsets<m
    n_mask=n_offsets<n
    mask=m_mask & n_mask

    aindx=inptr1+m_offsets*n+n_offsets
    bindx=inptr2+m_offsets*n+n_offsets
    cindx=outptr+m_offsets*n+n_offsets

    aval = tl.load(aindx, mask=mask)
    bval = tl.load(bindx, mask=mask)

    c=aval+bval
    tl.store(cindx,c,mask=mask)


In [12]:
def test():
    M, N = 512, 512
    a = torch.randn(M, N, device='cuda', dtype=torch.float32)
    b = torch.randn(M, N, device='cuda', dtype=torch.float32)
    c = torch.empty_like(a)
    BLOCK_M = 128
    BLOCK_N = 128
    grid_m = triton.cdiv(M, BLOCK_M)
    grid_n = triton.cdiv(N, BLOCK_N)
    
    matrixaddition[(grid_m, grid_n)](a, b, c,M, N,BLOCK_M,BLOCK_N)
    print(c)
    print(a+b)

    

In [13]:
if __name__=="__main__":
    test()

tensor([[-1.2546,  0.0445, -3.1837,  ...,  2.2120,  0.7079,  0.3060],
        [ 2.8690,  1.0473, -0.1397,  ..., -1.2675,  0.1488, -0.6596],
        [ 0.1879,  1.3677, -0.5178,  ..., -1.2892, -3.2989, -0.5914],
        ...,
        [-0.8974, -1.5136,  1.4986,  ...,  0.5035, -0.8137, -0.3021],
        [ 2.4848,  0.4959,  1.5156,  ...,  2.3876,  0.7809, -2.0143],
        [-2.2831, -0.4754,  2.1425,  ..., -1.1678, -1.5532,  1.1401]],
       device='cuda:0')
tensor([[-1.2546,  0.0445, -3.1837,  ...,  2.2120,  0.7079,  0.3060],
        [ 2.8690,  1.0473, -0.1397,  ..., -1.2675,  0.1488, -0.6596],
        [ 0.1879,  1.3677, -0.5178,  ..., -1.2892, -3.2989, -0.5914],
        ...,
        [-0.8974, -1.5136,  1.4986,  ...,  0.5035, -0.8137, -0.3021],
        [ 2.4848,  0.4959,  1.5156,  ...,  2.3876,  0.7809, -2.0143],
        [-2.2831, -0.4754,  2.1425,  ..., -1.1678, -1.5532,  1.1401]],
       device='cuda:0')
